# Analyze social media posts

In this notebook we will see how to build prompts fo analyze social media posts for our citizen repoting applications. The same prompts will be used in the main application code, in [NodeJS](../orchestrator) or [Python](../python_app).

### Prerequisites

Before starting:

* Make sure you have an AI Core instance with Generative AI Hub in place, and a running deployment for gpt-3.5-turbo. More details available [here](https://developers.sap.com/tutorials/ai-core-generative-ai.html).

* You should install the [SAP generative AI hub SDK](https://pypi.org/project/generative-ai-hub-sdk/). 

* After downloading the SAP generative AI hub SDK, you need to configure it to access your SAP AI Core instance. Instructions are available [here](https://pypi.org/project/generative-ai-hub-sdk/) under **Configuration**.

Check if everything has been set up correctly by running the prompt below:

In [2]:
from gen_ai_hub.proxy.langchain import ChatOpenAI
model = ChatOpenAI(  proxy_model_name='gpt-35-turbo' ,\
                    temperature=0.0,\
                    verbose = True)
model.invoke('What is your name?')

/Users/I550349/Library/CloudStorage/OneDrive-SAPSE/Documents/workspace/py_envs/gen-ai-hub/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


AIMessage(content="I am an AI language model developed by OpenAI, and I don't have a personal name. You can simply refer to me as OpenAI Assistant. How can I assist you today?")

### Engineer prompt to analyse social media post

Take a test post

In [3]:
test_post='''📢 Attention Sagenai residents! ⚠️

Can we talk about the disgraceful state of our neighborhood for a moment? 🗑️🤢 It seems like the local authorities have forgotten about our beloved public area on Oakwood Road. Seriously, has anyone seen the piles of rubbish and litter scattered everywhere? 🚯 It's like a landfill on our doorstep! I mean, who needs a clean and pleasant environment, right? 

📍 Oakwood Road, Sagenai

It's mildly infuriating how we pay our taxes and yet we have to put up with this filth! 🤬 I'm not asking for Buckingham Palace-like cleanliness, but a basic level of hygiene wouldn't hurt. Hopefully, the authorities will wake up from their slumber and do something about it ASAP. Let's keep our fingers crossed! 🤞

#CleanUpYourAct #OakwoodRoadNightmare #DisgustingNeighborhood 
 

        Coordinates:(51.57470453612761,0.003792117010085437)'''

Write all the information we need to get from the post with their description. This could be written in a configuration file.

In [4]:
info_dict={ 
"category": 
'''Classify the post in one of the following categories: \"PUBLIC CLEANLINESS\", \"ROADS & FOOTPATHS\", \
\"FACILITY & PARK MAINTENANCE\", \"PESTS\", \"DRAINS & SEWERS\".
If none of the categories fits, return \"OTHER\".''',                 

"priority": 
'''Identify the priority to be given to the reported issues into \"4-Low\", \"3-Medium\", \"2-High\", \"1-Very High\". .
    4-Low : the issue does not pose any problem with public safety and does not necessarily need to be handled urgently. 
    3-Medium : the issue does not cause any immediate danger, but it has significant and negative impact on the daily life of people in the neighborhood.
    2-High : the issue needs to be resolved quickly because it can potentially cause dangerous situations or disruptions. 
    1-Very High : the issue needs to be handled as soon as possible, as it is a matter of public safety. 
    ''',          
            
"summary": 
"Summarize the reported issue in 40 characters and a neutral tone.",
            
"description": 
"Summarize the reported issue in not more that 300 characters and a neutral tone.",

"address":
"Extract the address where the issue is taking place. Return the street only and omit the town or country",
            
"location": 
"Extract the coordinates where the issue has been notices. The format should be: float, float",
       
"sentiment" : 
'''Classify the sentiment of the post into \"NEUTRAL\", \"NEGATIVE\", \"VERY NEGATIVE\"
1. NEUTRAL: if the post reports an issue politely, in a calm tone
2. NEGATIVE: if the post shows irony, impatience, annoyance
3. VERY NEGATIVE: the post is rude or it expresses rage, hatred towards the public authority
'''
}


### Template approach 1 - more advanced

Here I am using advanced functionalities:

1. Langchain **template** to build the prompt template

1. Langchain **JSON output parser**, to parse the output in a JSON format

2. Langchain **chains** to concatenate operations, in our case the prompt template ---> model ---> output_parser

3. **OpenAI functions** : this is a functionality of certain openAI models, where you can pass a set of tools to the model that it can use to answer the prompt.


Create a template for the prompt to submit to the model. The template is super simple, and it just asks to extract information from the social media post.

In [5]:
from langchain.prompts import ChatPromptTemplate

template=''' Extract information from the social media post delimited by triple backticks.
```{post}``` 
'''

post_prompt= ChatPromptTemplate.from_template(template)

So, how do we specify exactly which information we want to get from the post?

In this case we are using OpenAI **Functions**. 

Certain OpenAI models (like gpt-3.5-turbo-0613 and gpt-4-0613) have been fine-tuned to detect when a function should be called and respond with the inputs that should be passed to the function. In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call those functions.

More info available here: https://learn.deeplearning.ai/functions-tools-agents-langchain/lesson/2/openai-function-calling. 

In [6]:
functions = [
    {
        "name": "post_analysis",
        "summary": "Extract information from the social media post",
        "parameters": {
            "type": "object",
            "properties": {
                
                "category": {"type": "string", "description": info_dict["category"], 
                             "enum":['PUBLIC CLEANLINESS','ROADS & FOOTPATHS','FACILITY & PARK MAINTENANCE',
                                     'PESTS', 'DRAINS & SEWERS','OTHER']},
                "priority": {"type": "string", "description": info_dict["priority"], 
                             "enum": ['4-Low','3-Medium', '2-High', '1-Very High'] },
                "summary": {"type": "string", "description": info_dict["summary"]},
                "description": {"type": "string", "description": info_dict["description"]},
                "address": {"type": "string", "description": info_dict["address"]},
                "location": {"type": "string", "description": info_dict["location"]},
                "sentiment": {"type": "string", "description": info_dict["sentiment"],
                              "enum": ['NEUTRAL','NEGATIVE', 'VERY NEGATIVE']},
            },
            "required": ["category", "priority"],
        },
    }
]


Let's input this to the model:

In [7]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    post_prompt
    | model.bind(function_call={"name": "post_analysis"}, functions=functions)
    | JsonOutputFunctionsParser()
)

chain.invoke({"post": test_post})

{'category': 'PUBLIC CLEANLINESS',
 'priority': '3-Medium',
 'summary': 'Disgraceful state of neighborhood',
 'description': "The public area on Oakwood Road is filled with piles of rubbish and litter scattered everywhere. It's like a landfill on our doorstep. We pay taxes and yet have to put up with this filth. Hoping the authorities will take action soon.",
 'address': 'Oakwood Road',
 'location': '51.57470453612761,0.003792117010085437',
 'sentiment': 'VERY NEGATIVE'}

This approach is said to be more reliable than just describing the info in a plain prompt 

### Template approach 2 - more simple

Here I am using just the langchain **prompt template** . 

In [8]:
template2='''SOCIAL MEDIA POST
{post}

INSTRUCTIONS 
For the social media post above, extract the following information: 
    
- category: {category}
    
- priority: {priority}

- summary: {summary}

- description: {description}
    
- address: {address}

- location: {location}

- sentiment: {sentiment}

Output a JSON file, all the fields should be in string format
'''

post_prompt2= ChatPromptTemplate.from_template(template2, )
model_input=post_prompt2.format_messages(post= test_post, 
              category=info_dict['category'],
              priority=info_dict['priority'],
              summary=info_dict['summary'],
              description=info_dict['description'],              
              address=info_dict['address'],
              location=info_dict['location'],
              sentiment=info_dict['sentiment'],
              )

model_output=model.invoke(model_input)
print(model_output.content)

{
  "category": "PUBLIC CLEANLINESS",
  "priority": "3-Medium",
  "summary": "Disgraceful state of neighborhood",
  "description": "The neighborhood on Oakwood Road is filled with piles of rubbish and litter, resembling a landfill. The local authorities seem to have forgotten about maintaining cleanliness in our beloved public area.",
  "address": "Oakwood Road",
  "location": "51.57470453612761,0.003792117010085437",
  "sentiment": "NEGATIVE"
}


#### Display exactly the prompt we sent to the model

In [9]:
print(model_input[0].content)

SOCIAL MEDIA POST
📢 Attention Sagenai residents! ⚠️

Can we talk about the disgraceful state of our neighborhood for a moment? 🗑️🤢 It seems like the local authorities have forgotten about our beloved public area on Oakwood Road. Seriously, has anyone seen the piles of rubbish and litter scattered everywhere? 🚯 It's like a landfill on our doorstep! I mean, who needs a clean and pleasant environment, right? 

📍 Oakwood Road, Sagenai

It's mildly infuriating how we pay our taxes and yet we have to put up with this filth! 🤬 I'm not asking for Buckingham Palace-like cleanliness, but a basic level of hygiene wouldn't hurt. Hopefully, the authorities will wake up from their slumber and do something about it ASAP. Let's keep our fingers crossed! 🤞

#CleanUpYourAct #OakwoodRoadNightmare #DisgustingNeighborhood 
 

        Coordinates:(51.57470453612761,0.003792117010085437)

INSTRUCTIONS 
For the social media post above, extract the following information: 
    
- category: Classify the post in 